## B. Setup & Package
### 1. Import library & penyeseuaian lingkungan notebook

Pada sel berikut saya impor library yang akan digunakan dan melakukan penyesuaian terhadap notebook saya.

In [26]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from sklearn.preprocessing import MinMaxScaler

### 2. Penyimpanan `PATH` file

In [7]:
raw_path = {
    'calendar.gz' : '../data/raw/calendar.csv.gz',
    'listings' : '../data/raw/listings.csv',
    'listings.gz' : '../data/raw/listings.csv.gz',
    'neighbourhoods' : '../data/raw/neighbourhoods.csv',
    'neighbourhoods.geo' : '../data/raw/neighbourhoods.geojson',
    'reviews' : '../data/raw/reviews.csv',
    'reviews.gz' : '../data/raw/reviews.csv.gz'
}

## C. Data Preparation
### 1. Baca dataset dari data yang ada sesuai **path**

In [25]:
calendar_gz = pd.read_csv(raw_path['calendar.gz'])

listings = pd.read_csv(raw_path['listings'])
listings_gz = pd.read_csv(raw_path['listings.gz'])

neighbourhoods = pd.read_csv(raw_path['neighbourhoods'])
neighbourhoods_geo = gpd.read_file(raw_path['neighbourhoods.geo'])

reviews = pd.read_csv(raw_path['reviews'])
reviews_gz = pd.read_csv(raw_path['reviews.gz'])

- Cek fragmen dataset yang duplikat!
    - `listings` dan `listings_gz`
    - `neighbourhoods` dan `neighbourhood_.geo`
    - `reviews` dan `reviews_gz`

In [ ]:
print("5 Kolom Pertama listings:", listings.head())
print("5 Kolom Pertama listings_gz:", listings_gz.head())

5 Kolom Pertama listings:        id                                              name  host_id  \
0   71609               Ensuite Room (Room 1 & 2) near EXPO   367042   
1   71896                   B&B  Room 1 near Airport & EXPO   367042   
2   71903                        Room 2-near Airport & EXPO   367042   
3  275343  10min walk to MRT & a Cozy Room with window! (1)  1439258   
4  275344             15 mins to Outram MRT Single Room (2)  1439258   

  host_name neighbourhood_group neighbourhood  latitude  longitude  \
0   Belinda         East Region      Tampines   1.34537  103.95887   
1   Belinda         East Region      Tampines   1.34754  103.95958   
2   Belinda         East Region      Tampines   1.34531  103.96100   
3       Kay      Central Region   Bukit Merah   1.29015  103.80814   
4       Kay      Central Region   Bukit Merah   1.28836  103.81144   

      room_type  price  minimum_nights  number_of_reviews last_review  \
0  Private room    NaN              92         

In [27]:
print("5 Kolom Pertama neighbourhoods:", neighbourhoods.head())
print("5 Kolom Pertama neighbourhoods:", neighbourhoods_geo.head())

5 Kolom Pertama neighbourhoods:   neighbourhood_group  neighbourhood
0      Central Region         Bishan
1      Central Region    Bukit Merah
2      Central Region    Bukit Timah
3      Central Region  Downtown Core
4      Central Region        Geylang
5 Kolom Pertama neighbourhoods:    neighbourhood neighbourhood_group  \
0      Pasir Ris         East Region   
1        Seletar   North-East Region   
2   Sungei Kadut        North Region   
3        Orchard      Central Region   
4  Downtown Core      Central Region   

                                            geometry  
0  MULTIPOLYGON (((103.95322 1.38202, 103.9535 1....  
1  MULTIPOLYGON (((103.88691 1.42649, 103.88812 1...  
2  MULTIPOLYGON (((103.7644 1.44345, 103.76443 1....  
3  MULTIPOLYGON (((103.84298 1.30001, 103.84294 1...  
4  MULTIPOLYGON (((103.85942 1.29978, 103.85954 1...  


In [28]:
print("5 Kolom Pertama reviews:", reviews.head())
print("5 Kolom Pertama reviews_gz:", reviews_gz.head())

5 Kolom Pertama reviews:    listing_id        date
0       71609  2011-12-19
1       71609  2012-07-17
2       71609  2012-09-01
3       71609  2012-09-04
4       71609  2013-01-02
5 Kolom Pertama reviews_gz:    listing_id       id        date  reviewer_id reviewer_name  \
0       71609   793880  2011-12-19      1456140           Max   
1       71609  1731810  2012-07-17      1804182         Jaiya   
2       71609  2162194  2012-09-01      3113461         Zahra   
3       71609  2190615  2012-09-04      1432123        Helmut   
4       71609  3221837  2013-01-02      2759938          Jack   

                                            comments  
0  The rooms were clean and tidy. Beds very comfo...  
1  Good space and quite an interesting home in a ...  
2  It was a comfortable place. Belinda was a kind...  
3  We are four mature age travellers and stayed f...  
4  Belinda is a great host! She helps you wheneve...  


Saya akan mengambil fragmen dataset:
- `listings.csv.gz`, 
- `neighbourhoods.geojson`,
- `reviews.csv.gz`,

serta akan mengabaikan:
- `listings.csv`,
- `neighbourhoods.csv`,
- `reviews.csv`

karena fragmen-fragmen dataset tersebut merupakan duplikat dan tidak komplit (yang diabaikan) dari versi yang akan saya ambil.

### 2. Proses *Cleaning* dan *Data Wrangling*

- Pengecekan untuk fragmen dataset `listings_gz` atau `listings.csv.gz`:

In [35]:
print("Head & Tail dari listings_gz:\n", listings_gz.head(), "\n", listings_gz.tail())

Head & Tail dari listings_gz:
        id                          listing_url       scrape_id last_scraped  \
0   71609   https://www.airbnb.com/rooms/71609  20250928035118   2025-09-28   
1   71896   https://www.airbnb.com/rooms/71896  20250928035118   2025-09-28   
2   71903   https://www.airbnb.com/rooms/71903  20250928035118   2025-09-28   
3  275343  https://www.airbnb.com/rooms/275343  20250928035118   2025-09-28   
4  275344  https://www.airbnb.com/rooms/275344  20250928035118   2025-09-28   

            source                                              name  \
0  previous scrape               Ensuite Room (Room 1 & 2) near EXPO   
1      city scrape                   B&B  Room 1 near Airport & EXPO   
2      city scrape                        Room 2-near Airport & EXPO   
3  previous scrape  10min walk to MRT & a Cozy Room with window! (1)   
4  previous scrape             15 mins to Outram MRT Single Room (2)   

                                         description  \
0    

- Identifikasi fragmen dataset di atas:
    - Kolom `price` di Airbnb biasanya mengandung simbol mata uang ($) dan operator desimal (,).
    - Kolom `bathrooms_text` berisi data kotor (contoh: "1.5 baths"). Saya ingin mengekstrak angkanya.
    - Ada *missing values* pada kolom `bedrooms`, `beds`, dan `bathrooms`. Prosedur penanganannya adalah imputasi menggunakan median, karena jumlah kamar/tempat tidur biasanya memiliki *outlier*.
    - Ada *missing values* pada kolom `price`. Prosedur penanganannya adalah imputasi berdasar median wilayah.

In [39]:
# 1. Pembersihan kolom Price
listings_gz['price'] = listings_gz['price'].replace(r'[\$,]','',regex=True).astype(float)

# 2. Ekstrak angka dari kolom bathrooms_text
listings_gz['bathrooms_count'] = listings_gz['bathrooms_text'].str.extract(r'(\d+\.?\d*)').astype(float)

# 3. Imputasi misval
listings_gz['bedrooms'] = listings_gz['bedrooms'].fillna(listings_gz['bedrooms'].median())
listings_gz['beds'] = listings_gz['beds'].fillna(listings_gz['beds'].median())
listings_gz['bathrooms_count'] = listings_gz['bathrooms_count'].fillna(listings_gz['bathrooms_count'].median())

# 4. Imputasi Harga berdasar median wilayah
listings_gz['price'] = listings_gz['price'].fillna(listings_gz.groupby('neighbourhood_cleansed')['price'].transform('median'))

- Standarisasi & Transformasi untuk fragmen dataset yang sama:
    - Harga Airbnb biasanya ada outlier (contoh: $10.000, kemungkinan salah ketik). Harus dibatasi batasan datanya.
    - Scaling data agar menyamakan skala fitur (*faeature engineering*).
    - Membuat **Host Seniority** agar `host_since` dapat diolah
    - Pengubahan biner untuk data 't' dan 'f' ke '1' dan '0'.

In [41]:
# 1. Handling Outlier
Q1 = listings_gz['price'].quantile(0.25)
Q3 = listings_gz['price'].quantile(0.75)
IQR = Q3 - Q1
upper_limit = Q3 + 1.5 * IQR
listings_gz['price_cleaned'] = listings_gz['price'].clip(0, upper_limit)

# 2. Scaling
scaler = MinMaxScaler()
listings_gz['price_scaled'] = scaler.fit_transform(listings_gz[['price_cleaned']])

# 3. Fitur "Host Seniority"
listings_gz['host_since'] = pd.to_datetime(listings_gz['host_since'])
listings_gz['host_seniority_years'] = 2025 - listings_gz['host_since'].dt.year

# 4. Binary Transformation
listings_gz['host_is_superhost'] = listings_gz['host_is_superhost'].map({'t': 1, 'f': 0}).fillna(0)

- Simpan fragmen dataset yang telah diolah.

In [45]:
columns_to_keep = [
    'id', 'name', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
    'latitude', 'longitude', 'room_type', 'price_cleaned', 'price_scaled',
    'bathrooms_count', 'bedrooms', 'beds', 'number_of_reviews', 
    'review_scores_rating', 'host_is_superhost', 'host_seniority_years'
]

listings_gz = listings_gz[columns_to_keep]
listings_gz.to_csv('../data/processed/listings_cleaned.csv', index=False)

- Pengecekan untuk fragmen dataset `calendar_gz` atau `calendar.csv.gz`:

In [40]:
print("Head & Tail dari calendar_gz:\n", calendar_gz.head(), "\n", calendar_gz.tail())

Head & Tail dari calendar_gz:
    listing_id        date available  price  adjusted_price  minimum_nights  \
0    12434512  2025-09-28         f    NaN             NaN              90   
1    12434512  2025-09-29         f    NaN             NaN              90   
2    12434512  2025-09-30         f    NaN             NaN              90   
3    12434512  2025-10-01         f    NaN             NaN              90   
4    12434512  2025-10-02         f    NaN             NaN              90   

   maximum_nights  
0            1125  
1            1125  
2            1125  
3            1125  
4            1125   
          listing_id        date available  price  adjusted_price  \
1347940    22395492  2026-09-23         t    NaN             NaN   
1347941    22395492  2026-09-24         t    NaN             NaN   
1347942    22395492  2026-09-25         t    NaN             NaN   
1347943    22395492  2026-09-26         t    NaN             NaN   
1347944    22395492  2026-09-27       

- Identifikasi fragmen dataset di atas:
    - Transformasi `date` ke **datetime**.

In [42]:
calendar_gz['date'] = pd.to_datetime(calendar_gz['date'])

- Standarisasi & Transformasi untuk fragmen dataset yang sama:
    - Fitur Ekstraksi Temporal, data tunggal sulit dianalisis untuk pola besar. Dapat dipecah menjadi bulanan.
    - Sinkronisasi Harga dari listings karena banyak kolom NaN.
    - Perlu melihat gambaran besar dari harga tahun 2025-2026 dengan agregasi bulanan. Jika tidak, nanti data terlalu *noisy* karena perharian.

In [48]:
# 1. Enrichment: Fitur Waktu
calendar_gz['month'] = calendar_gz['date'].dt.month
calendar_gz['day_name'] = calendar_gz['date'].dt.day_name()
calendar_gz['is_weekend'] = calendar_gz['date'].dt.weekday.isin([5, 6]).astype(int)

# 2. Sinkronisasi Harga dari Listings
price_map = listings_gz.set_index('id')['price_cleaned'].to_dict()
calendar_gz['price_final'] = calendar_gz['listing_id'].map(price_map)

# 3. Rataan harga bulanan
monthly_trend = calendar_gz.groupby('month')['price_final'].mean().reset_index()

- Simpan fragmen dataset yang telah diolah.

In [50]:
calendar_gz = calendar_gz[calendar_gz['available'] == 't'].copy()
calendar_gz.to_csv('../data/processed/calendar_cleaned.csv', index=False)

- Pengecekan untuk fragmen dataset `reviews_gz` atau `reviews.csv.gz`:

In [51]:
print("Head & Tail dari reviews_gz:\n", reviews_gz.head(), "\n", reviews_gz.tail())

Head & Tail dari reviews_gz:
    listing_id       id        date  reviewer_id reviewer_name  \
0       71609   793880  2011-12-19      1456140           Max   
1       71609  1731810  2012-07-17      1804182         Jaiya   
2       71609  2162194  2012-09-01      3113461         Zahra   
3       71609  2190615  2012-09-04      1432123        Helmut   
4       71609  3221837  2013-01-02      2759938          Jack   

                                            comments  
0  The rooms were clean and tidy. Beds very comfo...  
1  Good space and quite an interesting home in a ...  
2  It was a comfortable place. Belinda was a kind...  
3  We are four mature age travellers and stayed f...  
4  Belinda is a great host! She helps you wheneve...   
                 listing_id                   id        date  reviewer_id  \
38345  1486242516753075161  1495372723335409790  2025-08-25     79427140   
38346  1491162043816775904  1507048808726729363  2025-09-10    238798910   
38347  149566765692

- Identifikasi fragmen dataset di atas:
    - Ada reviu yang kosong, perlu dihapus.
    - Bersihkan tags HTML.
    - Mengubah format `date`.

In [52]:
# 1. Hapus reviu kosong
reviews_gz = reviews_gz.dropna(subset=['comments'])

# 2. Bersihkan tag HTML
reviews_gz['comments'] = reviews_gz['comments'].str.replace(r'<br\s*/?>', ' ', regex=True)

# 3. Standarisasi Tanggal
reviews_gz['date'] = pd.to_datetime(reviews_gz['date'])

- Standarisasi & Transformasi untuk fragmen dataset yang sama:
    - Fitur panjang karakter
    - Sinkronisasi Harga dari listings karena banyak kolom NaN.
    - Filtering ulasan yang dianggap spam (<3 karkter.)

In [53]:
# 1. Panjang karakter reviu
reviews_gz['review_length'] = reviews_gz['comments'].str.len()

# 2. Filtering: Ulasan terlalu pendek
reviews_gz = reviews_gz[reviews_gz['review_length'] > 3]

- Simpan fragmen dataset yang telah diolah.

In [54]:
reviews_gz.to_csv('../data/processed/reviews_cleaned.csv', index=False)

- Pengecekan untuk fragmen dataset `neighbourhoods_geo` atau `neighbourhoods.geojson`:

In [55]:
print("Head & Tail dari neighbourhoods_geo:\n", neighbourhoods_geo.head(), "\n", neighbourhoods_geo.tail())

Head & Tail dari neighbourhoods_geo:
    neighbourhood neighbourhood_group  \
0      Pasir Ris         East Region   
1        Seletar   North-East Region   
2   Sungei Kadut        North Region   
3        Orchard      Central Region   
4  Downtown Core      Central Region   

                                            geometry  
0  MULTIPOLYGON (((103.95322 1.38202, 103.9535 1....  
1  MULTIPOLYGON (((103.88691 1.42649, 103.88812 1...  
2  MULTIPOLYGON (((103.7644 1.44345, 103.76443 1....  
3  MULTIPOLYGON (((103.84298 1.30001, 103.84294 1...  
4  MULTIPOLYGON (((103.85942 1.29978, 103.85954 1...   
    neighbourhood neighbourhood_group  \
50       Hougang   North-East Region   
51     Serangoon   North-East Region   
52        Rochor      Central Region   
53   Marina East      Central Region   
54        Novena      Central Region   

                                             geometry  
50  MULTIPOLYGON (((103.91004 1.37586, 103.90985 1...  
51  MULTIPOLYGON (((103.87313 1.3873

- Identifikasi, standarisasi, transformasi fragmen dataset tersbut:

In [62]:
# 1. Load listing ke Geo-data
geo_names = neighbourhoods_geo['neighbourhood'].unique()
list_names = listings_gz['neighbourhood_cleansed'].unique()

# 2. Cek jika ada nama wilayah yang tidak cocok
mismatch = [name for name in list_names if name not in geo_names]
print("No. 2: Wilayah yang tidak sinkron:", mismatch)

# 3. Rataan harga per wilayah
price_agg = listings_gz.groupby('neighbourhood_cleansed')['price_cleaned'].mean().to_dict()

# 4. Inject data harga ke struktur GeoJSON
neighbourhoods_geo['avg_price'] = neighbourhoods_geo['neighbourhood'].map(price_agg).fillna(0)

# 5. Tambah metada properti per wilayah
count_agg = listings_gz['neighbourhood_cleansed'].value_counts().to_dict()
neighbourhoods_geo['listing_count'] = neighbourhoods_geo['neighbourhood'].map(count_agg).fillna(0)

No. 2: Wilayah yang tidak sinkron: []


- Simpan fragmen dataset yang telah diolah.

In [65]:
neighbourhoods_geo.to_file('../data/processed/neighbourhoods_cleaned.geojson', driver='GeoJSON')